# Value Iteration (5x5 Gridworld)

This notebook implements **Value Iteration** to solve a fully observable gridworld MDP.

**Environment**
- Grid: 5x5
- Actions: Up, Down, Left, Right
- Reward: +10 for reaching the goal, -1 otherwise
- Terminal: goal state
- Discount: γ = 0.9

**Output**
- Optimal value function \(V^*(s)\)
- Optimal policy \(\pi^*(s)\) after convergence

In [1]:
# Imports + Constants 
import numpy as np
from typing import Tuple, Dict, List

In [5]:
# MDP Setup 
# Grid size required by the question
GRID_SIZE = 5

# Discount factor required by the question
GAMMA = 0.9

# Convergence tolerance (small threshold for stopping condition)
THETA = 1e-6

# Define actions and deterministic movement deltas
ACTIONS = {
    "U": (-1, 0),
    "D": ( 1, 0),
    "L": ( 0,-1),
    "R": ( 0, 1),
}

# Choose a goal state (standard: bottom-right corner)
GOAL_STATE = (4, 4)

In [6]:
# Helpers + Transition Function 
def in_bounds(r: int, c: int) -> bool:
    # Checks whether a cell is inside the 5x5 grid
    return 0 <= r < GRID_SIZE and 0 <= c < GRID_SIZE

def all_states() -> List[Tuple[int,int]]:
    # Enumerates every state (r,c) in the grid
    return [(r, c) for r in range(GRID_SIZE) for c in range(GRID_SIZE)]

def step(state: Tuple[int,int], action: str) -> Tuple[Tuple[int,int], float, bool]:
    """
    Deterministic transition function.
    - If state is terminal (goal), remain there with 0 reward.
    - Otherwise move; if off-grid, stay in place.
    - Reward: +10 if next state is goal, else -1.
    - done=True if next state is goal.
    """
    if state == GOAL_STATE:
        return state, 0.0, True  # terminal absorbing
    
    dr, dc = ACTIONS[action]
    nr, nc = state[0] + dr, state[1] + dc

    # If move hits wall, agent stays in the same state
    if not in_bounds(nr, nc):
        nr, nc = state
    
    next_state = (nr, nc)

    # Reward rule from the question statement
    reward = 10.0 if next_state == GOAL_STATE else -1.0

    # Terminal condition
    done = (next_state == GOAL_STATE)
    return next_state, reward, done


In [7]:
# Value Iteration 
def value_iteration() -> Tuple[np.ndarray, Dict[Tuple[int,int], str]]:
    """
    Value Iteration:
        V_{k+1}(s) = max_a [ r(s,a,s') + gamma * V_k(s') ]
    Stops when max state change < THETA.
    """
    V = np.zeros((GRID_SIZE, GRID_SIZE), dtype=float)  # initialize V(s)=0

    while True:
        delta = 0.0  # max change in this iteration

        for s in all_states():
            if s == GOAL_STATE:
                continue  # keep terminal fixed
            
            old_v = V[s]  # store previous value

            # Compute one-step lookahead values for each action
            action_values = []
            for a in ACTIONS.keys():
                s_next, r, done = step(s, a)
                future = 0.0 if done else V[s_next]  # no future beyond terminal
                action_values.append(r + GAMMA * future)

            # Bellman optimality backup
            V[s] = max(action_values)

            # Update max change for convergence test
            delta = max(delta, abs(old_v - V[s]))

        if delta < THETA:
            break

    # Extract optimal greedy policy from converged V
    policy = {}
    for s in all_states():
        if s == GOAL_STATE:
            policy[s] = "G"
            continue
        
        best_a = None
        best_q = -float("inf")

        for a in ACTIONS.keys():
            s_next, r, done = step(s, a)
            future = 0.0 if done else V[s_next]
            q_sa = r + GAMMA * future

            if q_sa > best_q:
                best_q = q_sa
                best_a = a

        policy[s] = best_a

    return V, policy


In [8]:
# Print Results 
def print_policy(policy: Dict[Tuple[int,int], str]) -> None:
    arrow = {"U":"↑", "D":"↓", "L":"←", "R":"→", "G":"G"}
    print("\nOptimal Policy (Value Iteration):")
    for r in range(GRID_SIZE):
        print(" ".join(arrow[policy[(r,c)]] for c in range(GRID_SIZE)))

V_star, pi_star = value_iteration()

print("Optimal Value Function V* (rounded):")
print(np.round(V_star, 3))

print_policy(pi_star)

Optimal Value Function V* (rounded):
[[-0.434  0.629  1.81   3.122  4.58 ]
 [ 0.629  1.81   3.122  4.58   6.2  ]
 [ 1.81   3.122  4.58   6.2    8.   ]
 [ 3.122  4.58   6.2    8.    10.   ]
 [ 4.58   6.2    8.    10.     0.   ]]

Optimal Policy (Value Iteration):
↓ ↓ ↓ ↓ ↓
↓ ↓ ↓ ↓ ↓
↓ ↓ ↓ ↓ ↓
↓ ↓ ↓ ↓ ↓
→ → → → G
